In [107]:
import sys
import torch
import open3d as o3d
import os
import time
import numpy as np
import pandas as pd
import re

sys.path.insert(0,"../")

from utils import lddmm_utils, mesh_processing, viz

In [108]:
# torch type and device
use_cuda = torch.cuda.is_available()
torchdeviceId = torch.device("cuda:0") if use_cuda else "cpu"
torchdtype = torch.float32

# PyKeOps counterpart
KeOpsdeviceId = torchdeviceId.index  # id of Gpu device (in case Gpu is  used)
KeOpsdtype = torchdtype.__str__().split(".")[1]  # 'float32'

In [109]:
source_dir = "../data/preprocessed/"
data_paths = [source_dir + file for file in os.listdir(source_dir) if "ipynb" not in file]
data_paths

['../data/preprocessed/Canislupus_Oulu_preprocessed.ply',
 '../data/preprocessed/Canislupus_Bergen_preprocessed.ply',
 '../data/preprocessed/NHMO_preprocessed.ply',
 '../data/preprocessed/Canislupus_MZH_preprocessed.ply',
 '../data/preprocessed/Labrador_preprocessed.ply',
 '../data/preprocessed/NMB_preprocessed.ply',
 '../data/preprocessed/Canislupus_Lund_preprocessed.ply',
 '../data/preprocessed/bulldog_preprocessed.ply']

In [122]:
names = [re.search('../data/preprocessed/(.*)_preprocessed.ply', path).group(1) for path in data_paths]

In [123]:
names

['Canislupus_Oulu',
 'Canislupus_Bergen',
 'NHMO',
 'Canislupus_MZH',
 'Labrador',
 'NMB',
 'Canislupus_Lund',
 'bulldog']

In [112]:
def get_data(file):
    mesh = o3d.io.read_triangle_mesh(file)
    V, F, Rho = mesh_processing.getDataFromMesh(mesh)
    return(V,F,Rho)

list_data = []
for file in data_paths:
    list_data.append(get_data(file)[:2])

In [113]:
file_bulldog = "../data/preprocessed/bulldog_preprocessed.ply"
file_bergen = "../data/preprocessed/Canislupus_Bergen_preprocessed.ply"
file_labrador = "../data/preprocessed/Labrador_preprocessed.ply"
file_lund = "../data/preprocessed/Canislupus_Lund_preprocessed.ply"
file_MZH = "../data/preprocessed/Canislupus_MZH_preprocessed.ply"
file_oulu = "../data/preprocessed/Canislupus_Oulu_preprocessed.ply"
file_nhmo = "../data/preprocessed/NHMO_preprocessed.ply"
file_nmb = "../data/preprocessed/NMB_preprocessed.ply"

mesh_bulldog = o3d.io.read_triangle_mesh(file_bulldog)
V1, F1, Rho1 = mesh_processing.getDataFromMesh(mesh_bulldog)

mesh_bergen = o3d.io.read_triangle_mesh(file_bergen)
V2, F2, Rho2 = mesh_processing.getDataFromMesh(mesh_bergen)

mesh_labrador = o3d.io.read_triangle_mesh(file_labrador)
V3, F3, Rho3 = mesh_processing.getDataFromMesh(mesh_labrador)

mesh_lund = o3d.io.read_triangle_mesh(file_lund)
V4, F4, Rho4 = mesh_processing.getDataFromMesh(mesh_lund)

mesh_MZH = o3d.io.read_triangle_mesh(file_MZH)
V5, F5, Rho5 = mesh_processing.getDataFromMesh(mesh_MZH)

mesh_oulu = o3d.io.read_triangle_mesh(file_oulu)
V6, F6, Rho6 = mesh_processing.getDataFromMesh(mesh_oulu)

mesh_nhmo = o3d.io.read_triangle_mesh(file_nhmo)
V7, F7, Rho7 = mesh_processing.getDataFromMesh(mesh_nhmo)

mesh_nmb = o3d.io.read_triangle_mesh(file_nmb)
V8, F8, Rho8 = mesh_processing.getDataFromMesh(mesh_nmb)

In [114]:
#L = [[V1,F1],
 #    [V2,F2],
 #    [V3,F3],
 #    [V4,F4],
 #    [V5,F5],
 #    [V6,F6],
 #    [V7,F7],
 #    [V8,F8]]

In [115]:
#L = list_data

In [116]:
tab = []
for ind_s, ls in enumerate(list_data):
    C=[]
    padd = [0 for i in range(len(list_data) - ind_s)]
    for ind_t, lt in enumerate(list_data):
        if ind_s>ind_t:
        
            q0 = torch.from_numpy(ls[0]).clone().detach().to(dtype=torchdtype, device=torchdeviceId).requires_grad_(True)
            VT = torch.from_numpy(lt[0]).clone().detach().to(dtype=torchdtype, device=torchdeviceId)
            FS = torch.from_numpy(ls[1]).clone().detach().to(dtype=torch.long, device=torchdeviceId)
            FT = torch.from_numpy(lt[1]).clone().detach().to(dtype=torch.long, device=torchdeviceId)
            sigma = torch.tensor([10], dtype=torchdtype, device=torchdeviceId)

            x, y, z = (
                q0[:, 0].detach().cpu().numpy(),
                q0[:, 1].detach().cpu().numpy(),
                q0[:, 2].detach().cpu().numpy(),
            )
            i, j, k = (
                FS[:, 0].detach().cpu().numpy(),
                FS[:, 1].detach().cpu().numpy(),
                FS[:, 2].detach().cpu().numpy(),
            )

            xt, yt, zt = (
                VT[:, 0].detach().cpu().numpy(),
                VT[:, 1].detach().cpu().numpy(),
                VT[:, 2].detach().cpu().numpy(),
            )
            it, jt, kt = (
                FT[:, 0].detach().cpu().numpy(),
                FT[:, 1].detach().cpu().numpy(),
                FT[:, 2].detach().cpu().numpy(),
            )

#####################################################################
# Define data attachment and LDDMM functional

            dataloss = lddmm_utils.lossVarifoldSurf(FS, VT, FT, lddmm_utils.GaussLinKernel(sigma=sigma))
            Kv = lddmm_utils.GaussKernel(sigma=sigma)
            loss = lddmm_utils.LDDMMloss(Kv, dataloss)

######################################################################
# Perform optimization

# initialize momentum vectors
            p0 = torch.zeros(q0.shape, dtype=torchdtype, device=torchdeviceId, requires_grad=True)

            optimizer = torch.optim.LBFGS([p0], max_eval=10, max_iter=10)
            dist = int(loss(p0, q0).item()/1000)
            print("LDDM loss: ",ind_s, "-",ind_t, dist)
            C.append(dist)
            #print("Hausdorff distance: ", ind_s, "-",ind_t, hausdorff_distance(ls[0].detach().cpu().numpy(),lt[0].detach().cpu().numpy()))
    C = C + padd
    
    tab.append(C)

LDDM loss:  1 - 0 17011
LDDM loss:  2 - 0 87070
LDDM loss:  2 - 1 80237
LDDM loss:  3 - 0 45493
LDDM loss:  3 - 1 38105
LDDM loss:  3 - 2 94757
LDDM loss:  4 - 0 37021
LDDM loss:  4 - 1 32654
LDDM loss:  4 - 2 59591
LDDM loss:  4 - 3 48348
LDDM loss:  5 - 0 70166
LDDM loss:  5 - 1 64707
LDDM loss:  5 - 2 37717
LDDM loss:  5 - 3 86028
LDDM loss:  5 - 4 46186
LDDM loss:  6 - 0 53866
LDDM loss:  6 - 1 49506
LDDM loss:  6 - 2 117928
LDDM loss:  6 - 3 37199
LDDM loss:  6 - 4 59426
LDDM loss:  6 - 5 106478
LDDM loss:  7 - 0 85109
LDDM loss:  7 - 1 82469
LDDM loss:  7 - 2 85785
LDDM loss:  7 - 3 98986
LDDM loss:  7 - 4 73616
LDDM loss:  7 - 5 57528
LDDM loss:  7 - 6 125064


In [124]:
tab = np.array(tab)

In [125]:
tab

array([[     0,      0,      0,      0,      0,      0,      0,      0],
       [ 17011,      0,      0,      0,      0,      0,      0,      0],
       [ 87070,  80237,      0,      0,      0,      0,      0,      0],
       [ 45493,  38105,  94757,      0,      0,      0,      0,      0],
       [ 37021,  32654,  59591,  48348,      0,      0,      0,      0],
       [ 70166,  64707,  37717,  86028,  46186,      0,      0,      0],
       [ 53866,  49506, 117928,  37199,  59426, 106478,      0,      0],
       [ 85109,  82469,  85785,  98986,  73616,  57528, 125064,      0]])

In [126]:
df = pd.DataFrame(tab, index=names,columns=names)

In [127]:
df

,Canislupus_Oulu,Canislupus_Bergen,NHMO,Canislupus_MZH,Labrador,NMB,Canislupus_Lund,bulldog
Canislupus_Oulu,0,0,0,0,0,0,0,0
Canislupus_Bergen,17011,0,0,0,0,0,0,0
NHMO,87070,80237,0,0,0,0,0,0
Canislupus_MZH,45493,38105,94757,0,0,0,0,0
Labrador,37021,32654,59591,48348,0,0,0,0
NMB,70166,64707,37717,86028,46186,0,0,0
Canislupus_Lund,53866,49506,117928,37199,59426,106478,0,0
bulldog,85109,82469,85785,98986,73616,57528,125064,0


In [129]:
df.to_csv("doc/results/distance_varifold.csv")